In [ ]:
### Data Wrangling ###
import numpy as np
import pandas as pd

In [ ]:
#Tabnetモデル
#TN_sub = pd.read_csv('../input/fork2-tabnet-nn1ee0a1/submission_tab.csv')
#Tabnetモデルのnotebookの中で最も良いモデル
TN_sub = pd.read_csv('../input/moa-tabnet-inference/submission.csv')
#下は被っているため、NN2のnotebookと同じ人が更新したものであるため、モデルが似ているため保留
#NN_sub = pd.read_csv('../input/fork-of-tabnet-nn-4edfab/submission_NN.csv')
#non_scoredの予測をするモデル
NN2_sub = pd.read_csv('../input/fork2-tabnet-nn1ee0a1/NN2_submission.csv')
#drug_idを使って、転移学習をするモデル
NN3_sub = pd.read_csv('../input/pytorch-transfer-learning-with-k-folds-by-drug-ids/submission.csv')

In [ ]:
NN3_sub

In [ ]:
submission

In [ ]:
#各モデルの重みづけをする

#それぞれのモデルのpublicscoreで重みづけ
#public_score = [1-0.01841,1-0.01836,1-0.01835]
#wei_list = public_score

#それぞれのCV_lossで重みづけ
#wei_list = [1-0.016500671429559147, 1-0.014541765738774975,1-0.015610827671455082]

#重みなしの平均化
wei_list = [0.333333333333333,0.333333333333333,0.333333333333333]

model_weight = [wei_list[0]/sum(wei_list),wei_list[0]/sum(wei_list),wei_list[0]/sum(wei_list)]

In [ ]:
model_weight

In [ ]:
submission = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')

public_ids = list(submission['sig_id'].values)

submission.shape

In [ ]:
test = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')

test_ids = list(test['sig_id'].values)

private_ids = list(set(test_ids)-set(public_ids))

len(private_ids)

In [ ]:
#モデルのリストを作る
predictions = [TN_sub, NN2_sub, NN3_sub]

In [ ]:
predictions[0]

In [ ]:
#sampleの形を参照する
target_columns = list(submission.columns)
target_columns.remove('sig_id')

In [ ]:
#sampleの形を参照する
#target_columns = list(submission.columns)
#target_columns.remove('sig_id')
#空の予測値データフレームを作る
y_pred = pd.DataFrame()
y_pred['sig_id'] = predictions[0]['sig_id']

for column in target_columns:
    #任意のターゲット列「x列」に着目する
    column_data = []
    for i in range(len(predictions)):
        #各モデルのx列（n行*1列）をcolumn_dataの中に入れる
        #これをモデルの数だけ繰り返しn行*2列まで行う <-- update add model_weight
        column_data.append(predictions[i][column])
    #横方向で平均をとりn行*1列にする <-- update mean->sum
    y_pred[column] = np.mean(column_data, axis=0)

y_pred.shape

In [ ]:
#submission = pd.DataFrame(index = public_ids + private_ids, columns=target_columns)
#submission.index.name = 'sig_id'

#submission[:] = 0

#submission.loc[y_pred.sig_id,:] = y_pred[target_columns].values

#submission = np.clip(submission,0.0001,0.9999)

#submission.loc[test[test.cp_type == 'ctl_vehicle'].sig_id] = 0

submission = pd.DataFrame(index = public_ids + private_ids, columns=target_columns)
submission.index.name = 'sig_id'
submission[:] = 0
for column in target_columns:
    submission[column] = np.array(y_pred[column])

submission = np.clip(submission,0.0001,0.9999)
submission.loc[test[test.cp_type == 'ctl_vehicle'].sig_id] = 0

#1モデル
submission.to_csv('submission.csv', index=True)


#submission.to_csv('submission.csv', index=True)

In [ ]:
submission.iloc[:,0] = np.array(y_pred["5-alpha_reductase_inhibitor"])

In [ ]:
submission